In [1]:
import sys

sys.path.append(r"C:\\Users\\a-sochat\\development\\pysweep2.0")

import time 

import numpy as np
import itertools

from pysweep import sweep_object, sweep_zip, sweep_product, BaseSweepObject
import pysweep.utils 
from pysweep.tests.testing_utilities import ParameterFactory

import qcodes

<IPython.core.display.Javascript object>

In [2]:
param_factory = ParameterFactory(sys.stdout)  # This simply gives us QCoDeS parameters which, when set, will write 
# logs to stdout 

In [3]:
# For example
p = param_factory[0]

print(isinstance(p, qcodes.StandardParameter))

p.set(10)

True
setting parameter 0 to 10

In [4]:
# Lets test sweeping over a parameter
sweep_values = [0, 1, 2, 3]

for log_line in sweep_object(p, sweep_values):
    print("\nlogline = {}".format(log_line))

setting parameter 0 to 0
logline = {'parameter 0': {'unit': 'V', 'value': 0}}
setting parameter 0 to 1
logline = {'parameter 0': {'unit': 'V', 'value': 1}}
setting parameter 0 to 2
logline = {'parameter 0': {'unit': 'V', 'value': 2}}
setting parameter 0 to 3
logline = {'parameter 0': {'unit': 'V', 'value': 3}}


We see that: 
* We can loop over a sweep object (that is, \_\_iter\_\_ and \_\_next\_\_ are implemented) 
* At every iteration the independent experiment variables are set
* At every iteration a dictionary is returned, which are meant to be included in the experiment output file 

In [5]:
# We can also sweep over a function, instead of a QCoDeS parameter. This can be handy to make a time series measurement

def timer():
    t0 = time.time()
    def inner(station, namespace, t):  # Everytime this function is called, log the current time  
        return {"time": {"unit": "s", "value": "{:.2}".format(time.time() - t0)}}
    return inner

for log_line in sweep_object(timer(), range(10)).after_each(pysweep.utils.sleep(.1)):
    print("logline = {}".format(log_line))

logline = {'time': {'unit': 's', 'value': '0.0'}}
logline = {'time': {'unit': 's', 'value': '0.1'}}
logline = {'time': {'unit': 's', 'value': '0.2'}}
logline = {'time': {'unit': 's', 'value': '0.3'}}
logline = {'time': {'unit': 's', 'value': '0.4'}}
logline = {'time': {'unit': 's', 'value': '0.5'}}
logline = {'time': {'unit': 's', 'value': '0.6'}}
logline = {'time': {'unit': 's', 'value': '0.71'}}
logline = {'time': {'unit': 's', 'value': '0.81'}}
logline = {'time': {'unit': 's', 'value': '0.91'}}


The "after each" command tells the sweep object to execute some command after calling the timer function (or setting some 
parameter)

In [6]:
# Now suppose we want to set parameters and log the time at the same time

for log_line in sweep_zip(
    sweep_object(timer(), itertools.repeat(1)).after_each(pysweep.utils.sleep(.1)), 
    sweep_object(p, sweep_values)
):
    print("\nlogline = {}".format(log_line))

setting parameter 0 to 0
logline = {'time': {'unit': 's', 'value': '0.0'}, 'parameter 0': {'unit': 'V', 'value': 0}}
setting parameter 0 to 1
logline = {'time': {'unit': 's', 'value': '0.1'}, 'parameter 0': {'unit': 'V', 'value': 1}}
setting parameter 0 to 2
logline = {'time': {'unit': 's', 'value': '0.2'}, 'parameter 0': {'unit': 'V', 'value': 2}}
setting parameter 0 to 3
logline = {'time': {'unit': 's', 'value': '0.3'}, 'parameter 0': {'unit': 'V', 'value': 3}}


The "sweep_zip" command lets us perform two sweeps simultaniously 

In [7]:
# We can accomplish the same with a sligtly cleaner syntax. First we define some aliases: 

def log_time():
    tb = time.time()

    def inner(station, namespace):
        return {"time": {"unit": "s", "value": "{:.2}".format(time.time() - tb)}}
    return inner

BaseSweepObject.add_alias("log_time", lambda so: so.after_each(log_time()))
BaseSweepObject.add_alias("sleep", lambda so, t: so.after_each(pysweep.utils.sleep(t)))

In [8]:
# Now the magic happens

for log_line in sweep_object(p, sweep_values).sleep(.1).log_time(): 
    print("\nlogline = {}".format(log_line))

setting parameter 0 to 0
logline = {'parameter 0': {'unit': 'V', 'value': 0}, 'time': {'unit': 's', 'value': '0.1'}}
setting parameter 0 to 1
logline = {'parameter 0': {'unit': 'V', 'value': 1}, 'time': {'unit': 's', 'value': '0.2'}}
setting parameter 0 to 2
logline = {'parameter 0': {'unit': 'V', 'value': 2}, 'time': {'unit': 's', 'value': '0.3'}}
setting parameter 0 to 3
logline = {'parameter 0': {'unit': 'V', 'value': 3}, 'time': {'unit': 's', 'value': '0.4'}}


We see that we get the same result as before 

Now lets suppose we want to set a parameter to a certain value and then take a time series experiment. Lets create a 
fictitious instrument representing a noisy ohmic resistor

In [9]:
class NoisyResistor(qcodes.Instrument): 
    def __init__(self, name, resistance):
        super().__init__(name)
    
        self._resistance = resistance
        self._bias = 0
    
        self.add_parameter(
            name="bias", 
            set_cmd=self._set_bias,
            units="V"
        )
        
        self.add_parameter(
            name="current", 
            get_cmd=self._get_current, 
            units="A"
        )
        
        self.connect_message()
        
    def _set_bias(self, bias): 
        self._bias = bias
    
    def _get_current(self): 
        return self._bias / self._resistance + np.random.normal(0.0, 0.1)
    
    def connect_message(self, idn_param='IDN', begin_time=None): 
        
        con_msg = ('Connected to: noisy resistor')
        print(con_msg)
        return {}

r = NoisyResistor("R", 10)

Connected to: noisy resistor


In [10]:
station = qcodes.Station(r)

In [11]:
def measure_current(station, namespace, _): 
    return {station.R.current.label: {"unit": station.R.current.units, "value": "{:.3}".format(station.R.current())}}
    
for log_line in sweep_object(measure_current, 10 * [1]).set_station(station).sleep(.1).log_time():
    print("logline = {}".format(log_line))

logline = {'current': {'unit': 'A', 'value': '-0.0754'}, 'time': {'unit': 's', 'value': '0.1'}}
logline = {'current': {'unit': 'A', 'value': '0.00231'}, 'time': {'unit': 's', 'value': '0.2'}}
logline = {'current': {'unit': 'A', 'value': '0.0618'}, 'time': {'unit': 's', 'value': '0.3'}}
logline = {'current': {'unit': 'A', 'value': '-0.0806'}, 'time': {'unit': 's', 'value': '0.4'}}
logline = {'current': {'unit': 'A', 'value': '-0.00398'}, 'time': {'unit': 's', 'value': '0.5'}}
logline = {'current': {'unit': 'A', 'value': '0.25'}, 'time': {'unit': 's', 'value': '0.61'}}
logline = {'current': {'unit': 'A', 'value': '-0.0739'}, 'time': {'unit': 's', 'value': '0.71'}}
logline = {'current': {'unit': 'A', 'value': '0.0418'}, 'time': {'unit': 's', 'value': '0.81'}}
logline = {'current': {'unit': 'A', 'value': '-0.00876'}, 'time': {'unit': 's', 'value': '0.91'}}
logline = {'current': {'unit': 'A', 'value': '-0.149'}, 'time': {'unit': 's', 'value': '1.0'}}


In [12]:
# Now lets vary the bias voltage and taking a time trace at each set point 

for log_line in sweep_product(
    sweep_object(measure_current, 3 * [1]).set_station(station).sleep(.1).log_time(), 
    sweep_object(r.bias, [0, 1, 2])
):
    print("logline = {}".format(log_line))

logline = {'current': {'unit': 'A', 'value': '0.00781'}, 'time': {'unit': 's', 'value': '0.1'}, 'bias': {'unit': 'V', 'value': 0}}
logline = {'current': {'unit': 'A', 'value': '-0.0942'}, 'time': {'unit': 's', 'value': '0.2'}, 'bias': {'unit': 'V', 'value': 0}}
logline = {'current': {'unit': 'A', 'value': '0.0458'}, 'time': {'unit': 's', 'value': '0.3'}, 'bias': {'unit': 'V', 'value': 0}}
logline = {'current': {'unit': 'A', 'value': '-0.000905'}, 'time': {'unit': 's', 'value': '0.4'}, 'bias': {'unit': 'V', 'value': 1}}
logline = {'current': {'unit': 'A', 'value': '0.117'}, 'time': {'unit': 's', 'value': '0.5'}, 'bias': {'unit': 'V', 'value': 1}}
logline = {'current': {'unit': 'A', 'value': '0.145'}, 'time': {'unit': 's', 'value': '0.6'}, 'bias': {'unit': 'V', 'value': 1}}
logline = {'current': {'unit': 'A', 'value': '0.177'}, 'time': {'unit': 's', 'value': '0.71'}, 'bias': {'unit': 'V', 'value': 2}}
logline = {'current': {'unit': 'A', 'value': '0.388'}, 'time': {'unit': 's', 'value': '

In [13]:
for log_line in sweep_product(
    sweep_object(measure_current, 3 * [1]).sleep(1).log_time(), 
    sweep_object(r.bias, [0, 1, 2])
).set_station(station):
    print("logline = {}".format(log_line))

logline = {'current': {'unit': 'A', 'value': '-0.0551'}, 'time': {'unit': 's', 'value': '1.0'}, 'bias': {'unit': 'V', 'value': 0}}
logline = {'current': {'unit': 'A', 'value': '0.209'}, 'time': {'unit': 's', 'value': '2.0'}, 'bias': {'unit': 'V', 'value': 0}}
logline = {'current': {'unit': 'A', 'value': '-0.121'}, 'time': {'unit': 's', 'value': '3.0'}, 'bias': {'unit': 'V', 'value': 0}}
logline = {'current': {'unit': 'A', 'value': '0.0222'}, 'time': {'unit': 's', 'value': '4.0'}, 'bias': {'unit': 'V', 'value': 1}}
logline = {'current': {'unit': 'A', 'value': '0.0909'}, 'time': {'unit': 's', 'value': '5.0'}, 'bias': {'unit': 'V', 'value': 1}}
logline = {'current': {'unit': 'A', 'value': '-0.0142'}, 'time': {'unit': 's', 'value': '6.0'}, 'bias': {'unit': 'V', 'value': 1}}
logline = {'current': {'unit': 'A', 'value': '0.292'}, 'time': {'unit': 's', 'value': '7.0'}, 'bias': {'unit': 'V', 'value': 2}}
logline = {'current': {'unit': 'A', 'value': '0.0313'}, 'time': {'unit': 's', 'value': '8.

ModuleNotFoundError: No module named 'jsonmerge'